# Execute the trained model (reduced size)

In [3]:
import os
import sys

current_path = os.path.abspath('.')
root_path = os.path.dirname(os.path.dirname(current_path))
sys.path.append(root_path)

from sourcecode.BONE_CHANNELS.bones_train_512x512 import *
from sourcecode.BONE_CHANNELS.bones_dataloader_512x512 import *

import torchvision.transforms.functional as TF
from torchvision import transforms
from torchvision import utils
from datetime import datetime

from scipy import ndimage as nd
from skimage import measure

dataset_dir = "C:\\Igor\\dali\\tumor_regions_segmentation-main\\datasets\\BONE_CHANNELS_reduced_size"
model_dir = "../../models"

batch_size = 1
patch_size = (512, 512)
color_model = "LAB"
dataloaders = create_dataloader(batch_size=batch_size, 
                                shuffle=False,
                                dataset_dir=dataset_dir,
                                color_model=color_model)

dataset_train_size = len(dataloaders['train'].dataset)
dataset_test_size = len(dataloaders['test'].dataset)

logger.info('Train dataset size: {}'.format(dataset_train_size))
logger.info('Test dataset size: {}'.format(dataset_test_size))

threshold_prob = 0.50
threshold_itc = 200/(0.243 * pow(2, 5))


# Checking for GPU availability
use_cuda=True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if use_cuda else "cpu"
logger.info('Runing on: {}'.format(device))

# loads our trained fcn model
trained_model_version = "BONE_CHANNELS_reduced_size__Size-512x512_Epoch-10_Images-9_Batch-1__random_0_operations_all"
trained_model_path="{}/{}".format(model_dir, '{}.pth'.format(trained_model_version))
model = load_checkpoint(file_path=trained_model_path, img_input_size=patch_size, use_cuda=use_cuda)
cont = 0
for batch_idx, (images, masks, fname, original_size) in enumerate(dataloaders['test']):
        
    X = Variable(images).to(device) if use_cuda else images
    logger.info('Batch {}: {}/{} images: {} masks: {} {}'.format(
                (batch_idx+1), 
                (batch_idx+1) * len(images), 
                len(dataloaders['test'].dataset),
                images.shape,
                masks.shape,
                datetime.now().strftime('%d/%m/%Y %H:%M:%S')))
    
    #X_numpy = X.cpu().numpy()
    y_hat = model(X).detach().cpu().squeeze(0)
    cls = "bones"
    
    input_image = transforms.ToPILImage()(X.squeeze(0).cpu())
    output_mask = basic_threshold(y_hat[0].detach().cpu().numpy(), threshold=threshold_prob, output_type="uint8")

    predicted_labels = measure.label(output_mask, connectivity=2)
    output_mask = np.zeros((output_mask.shape[0], output_mask.shape[1]))
    labels = np.unique(predicted_labels)
    properties = measure.regionprops(predicted_labels)
    for lbl in range(1, (np.max(labels)+1)):
        major_axis_length = properties[lbl-1].major_axis_length
        if major_axis_length > threshold_itc:
            output_mask[predicted_labels == lbl] = 1

    input_image_rgb = lab_to_rgb(pil_to_np(input_image))
    roi_image = blend_image(np_to_pil(input_image_rgb), np_to_pil(output_mask), foreground='red', alpha=0.6, inverse=True)

    # results dir
    logger.info('Fname: {}'.format(fname))
    wsi_image_number = fname[0].split(".")[0]
    patch_images_results_dir = "{}\\results\\{}\\testing\\{}\\patch\\{}x{}\\{}".format(dataset_dir, trained_model_version, cls, patch_size[0], patch_size[1], wsi_image_number)
    results_output_dir = "{}\\01-unet_result".format(patch_images_results_dir)
    if not os.path.exists(results_output_dir):
        os.makedirs(results_output_dir)
        
    results_roi_dir = "{}\\02-roi".format(patch_images_results_dir)
    if not os.path.exists(results_roi_dir):
        os.makedirs(results_roi_dir)

    # save the results
    patch_img_name = fname[0]
    utils.save_image(y_hat[0], '{}\\{}'.format(results_output_dir, patch_img_name))
    utils.save_image(TF.to_tensor(roi_image), '{}\\{}'.format(results_roi_dir, patch_img_name))

2023-04-24 18:49:27,039 :: INFO load_dataset :: [training] C:\Igor\dali\tumor_regions_segmentation-main\datasets\BONE_CHANNELS_reduced_size/training
2023-04-24 18:49:27,043 :: INFO load_dataset :: C:\Igor\dali\tumor_regions_segmentation-main\datasets\BONE_CHANNELS_reduced_size/training/bones/301.png
2023-04-24 18:49:27,043 :: INFO load_dataset :: C:\Igor\dali\tumor_regions_segmentation-main\datasets\BONE_CHANNELS_reduced_size/training/bones/302.png
2023-04-24 18:49:27,044 :: INFO load_dataset :: C:\Igor\dali\tumor_regions_segmentation-main\datasets\BONE_CHANNELS_reduced_size/training/bones/305.png
2023-04-24 18:49:27,045 :: INFO load_dataset :: C:\Igor\dali\tumor_regions_segmentation-main\datasets\BONE_CHANNELS_reduced_size/training/bones/306.png
2023-04-24 18:49:27,045 :: INFO load_dataset :: C:\Igor\dali\tumor_regions_segmentation-main\datasets\BONE_CHANNELS_reduced_size/training/bones/310.png
2023-04-24 18:49:27,047 :: INFO load_dataset :: C:\Igor\dali\tumor_regions_segmentation-mai

In [ ]:
import os
import sys

current_path = os.path.abspath('.')
root_path = os.path.dirname(os.path.dirname(current_path))
sys.path.append(root_path)

from sourcecode.BONE_CHANNELS.bones_train import *
from sourcecode.BONE_CHANNELS.bones_dataloader_reduced_size import *

import torchvision.transforms.functional as TF
from torchvision import transforms
from torchvision import utils
from datetime import datetime

from scipy import ndimage as nd
from skimage import measure

dataset_dir = "../../datasets/BONE_CHANNELS_reduced_size"
model_dir = "../../models"

batch_size = 1
patch_size = (512, 512)
color_model = "LAB"
dataloaders = create_dataloader(batch_size=batch_size, 
                                shuffle=False,
                                dataset_dir=dataset_dir,
                                color_model=color_model)

dataset_train_size = len(dataloaders['train'].dataset)
dataset_test_size = len(dataloaders['test'].dataset)


threshold_prob = 0.50
threshold_itc = 200/(0.243 * pow(2, 5))


# Checking for GPU availability
use_cuda=True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if use_cuda else "cpu"
logger.info('Runing on: {}'.format(device))

# loads our trained fcn model
trained_model_version = "BONE_CHANNELS_reduced_size__Size-reduced_size_Epoch-400_Images-100_Batch-1__standard_9_operations_all"
trained_model_path="{}/{}".format(model_dir, '{}.pth'.format(trained_model_version))
model = load_checkpoint(file_path=trained_model_path, img_input_size=patch_size, use_cuda=use_cuda)
cont = 0
for batch_idx, (images, masks, fname, original_size) in enumerate(dataloaders['test']):
        
    X = Variable(images).to(device) if use_cuda else images
    logger.info('Batch {}: {}/{} images: {} masks: {} {}'.format(
                (batch_idx+1), 
                (batch_idx+1) * len(images), 
                len(dataloaders['test'].dataset),
                images.shape,
                masks.shape,
                datetime.now().strftime('%d/%m/%Y %H:%M:%S')))
    
    #X_numpy = X.cpu().numpy()
    y_hat = model(X).detach().cpu().squeeze(0)
    cls = "tumor"
    
    input_image = transforms.ToPILImage()(X.squeeze(0).cpu())
    output_mask = basic_threshold(y_hat[0].detach().cpu().numpy(), threshold=threshold_prob, output_type="uint8")

    predicted_labels = measure.label(output_mask, connectivity=2)
    output_mask = np.zeros((output_mask.shape[0], output_mask.shape[1]))
    labels = np.unique(predicted_labels)
    properties = measure.regionprops(predicted_labels)
    for lbl in range(1, (np.max(labels)+1)):
        major_axis_length = properties[lbl-1].major_axis_length
        if major_axis_length > threshold_itc:
            output_mask[predicted_labels == lbl] = 1

    input_image_rgb = lab_to_rgb(pil_to_np(input_image))
    roi_image = blend_image(np_to_pil(input_image_rgb), np_to_pil(output_mask), foreground='red', alpha=0.6, inverse=True)

    # results dir
    wsi_image_number = fname[0].split("_")[0] + "_" + fname[0].split("_")[1]
    patch_images_results_dir = "{}/results/{}/testing/{}/patch/{}x{}/{}".format(dataset_dir, trained_model_version, cls, patch_size[0], patch_size[1], wsi_image_number)
    results_output_dir = "{}/01-unet_result".format(patch_images_results_dir)
    if not os.path.exists(results_output_dir):
        os.makedirs(results_output_dir)
        
    results_roi_dir = "{}/02-roi".format(patch_images_results_dir)
    if not os.path.exists(results_roi_dir):
        os.makedirs(results_roi_dir)

    # save the results
    patch_img_name = fname[0]
    utils.save_image(y_hat[0], '{}/{}'.format(results_output_dir, patch_img_name))
    utils.save_image(TF.to_tensor(roi_image), '{}/{}'.format(results_roi_dir, patch_img_name))

In [ ]:
import os
import sys

current_path = os.path.abspath('.')
root_path = os.path.dirname(os.path.dirname(current_path))
sys.path.append(root_path)

from sourcecode.BONE_CHANNELS.bones_train import *
from sourcecode.BONE_CHANNELS.bones_dataloader_reduced_size import *

import torchvision.transforms.functional as TF
from torchvision import transforms
from torchvision import utils
from datetime import datetime

from scipy import ndimage as nd
from skimage import measure

dataset_dir = "../../datasets/BONE_CHANNELS_reduced_size"
model_dir = "../../models"

batch_size = 1
patch_size = (512, 512)
color_model = "LAB"
dataloaders = create_dataloader(batch_size=batch_size, 
                                shuffle=False,
                                dataset_dir=dataset_dir,
                                color_model=color_model)

dataset_train_size = len(dataloaders['train'].dataset)
dataset_test_size = len(dataloaders['test'].dataset)


threshold_prob = 0.50
threshold_itc = 200/(0.243 * pow(2, 5))


# Checking for GPU availability
use_cuda=True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if use_cuda else "cpu"
logger.info('Runing on: {}'.format(device))

# loads our trained fcn model
trained_model_version = "BONE_CHANNELS_reduced_size__Size-reduced_size_Epoch-400_Images-100_Batch-1__inpainting"
trained_model_path="{}/{}".format(model_dir, '{}.pth'.format(trained_model_version))
model = load_checkpoint(file_path=trained_model_path, img_input_size=patch_size, use_cuda=use_cuda)
cont = 0
for batch_idx, (images, masks, fname, original_size) in enumerate(dataloaders['test']):
        
    X = Variable(images).to(device) if use_cuda else images
    logger.info('Batch {}: {}/{} images: {} masks: {} {}'.format(
                (batch_idx+1), 
                (batch_idx+1) * len(images), 
                len(dataloaders['test'].dataset),
                images.shape,
                masks.shape,
                datetime.now().strftime('%d/%m/%Y %H:%M:%S')))
    
    #X_numpy = X.cpu().numpy()
    y_hat = model(X).detach().cpu().squeeze(0)
    cls = "tumor"
    
    input_image = transforms.ToPILImage()(X.squeeze(0).cpu())
    output_mask = basic_threshold(y_hat[0].detach().cpu().numpy(), threshold=threshold_prob, output_type="uint8")

    predicted_labels = measure.label(output_mask, connectivity=2)
    output_mask = np.zeros((output_mask.shape[0], output_mask.shape[1]))
    labels = np.unique(predicted_labels)
    properties = measure.regionprops(predicted_labels)
    for lbl in range(1, (np.max(labels)+1)):
        major_axis_length = properties[lbl-1].major_axis_length
        if major_axis_length > threshold_itc:
            output_mask[predicted_labels == lbl] = 1

    input_image_rgb = lab_to_rgb(pil_to_np(input_image))
    roi_image = blend_image(np_to_pil(input_image_rgb), np_to_pil(output_mask), foreground='red', alpha=0.6, inverse=True)

    # results dir
    wsi_image_number = fname[0].split("_")[0] + "_" + fname[0].split("_")[1]
    patch_images_results_dir = "{}/results/{}/testing/{}/patch/{}x{}/{}".format(dataset_dir, trained_model_version, cls, patch_size[0], patch_size[1], wsi_image_number)
    results_output_dir = "{}/01-unet_result".format(patch_images_results_dir)
    if not os.path.exists(results_output_dir):
        os.makedirs(results_output_dir)
        
    results_roi_dir = "{}/02-roi".format(patch_images_results_dir)
    if not os.path.exists(results_roi_dir):
        os.makedirs(results_roi_dir)

    # save the results
    patch_img_name = fname[0]
    utils.save_image(y_hat[0], '{}/{}'.format(results_output_dir, patch_img_name))
    utils.save_image(TF.to_tensor(roi_image), '{}/{}'.format(results_roi_dir, patch_img_name))